# Crypto Intelligence: Top 50 Cryptocurrency Analysis

This notebook demonstrates an end-to-end data engineering and analytics pipeline to analyze and rank the **top 50 cryptocurrencies** based on historical prices, technical indicators (SMA, RSI), and simulated sentiment data.

### Features:
- Dynamic top 50 crypto selection via CoinGecko API
- Price data download using yfinance
- SMA & RSI technical indicators
- Weighted ranking algorithm including sentiment scores
- Interactive visualization using Plotly
- Markdown documentation embedded for clear methodology explanation

In [ ]:
# Install required packages
!pip install yfinance pandas numpy plotly requests seaborn

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import requests
import plotly.express as px
import seaborn as sns
sns.set_style('whitegrid')

## Step 1: Fetch Top 50 Cryptocurrencies from CoinGecko

In [ ]:
def get_top_cryptos(limit=50):
    url = 'https://api.coingecko.com/api/v3/coins/markets'
    params = {'vs_currency':'usd','order':'market_cap_desc','per_page':limit,'page':1}
    response = requests.get(url, params=params).json()
    return [coin['symbol'].upper() for coin in response]

tickers = get_top_cryptos(50)
tickers[:10]  # show first 10 for sanity check

## Step 2: Download Historical Price Data

In [ ]:
def get_crypto_prices(tickers, start='2023-01-01', end='2025-12-31'):
    data = {}
    for ticker in tickers:
        try:
            df = yf.download(ticker+'-USD', start=start, end=end)
            data[ticker] = df['Close']
        except:
            continue
    return pd.DataFrame(data)

prices = get_crypto_prices(tickers)
prices.head()

## Step 3: Technical Indicator Functions

In [ ]:
def calculate_sma(df, window=20):
    return df.rolling(window=window).mean()

def calculate_rsi(df, window=14):
    delta = df.diff()
    gain = delta.clip(lower=0)
    loss = -1 * delta.clip(upper=0)
    avg_gain = gain.rolling(window=window).mean()
    avg_loss = loss.rolling(window=window).mean()
    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

## Step 4: Simulated Sentiment Scores

In [ ]:
# Generate random sentiment scores between 0.4 and 1.0 for demonstration
np.random.seed(42)
sentiment_scores = {ticker: np.random.uniform(0.4,1.0) for ticker in prices.columns}
sentiment_scores

## Step 5: Ranking Algorithm

In [ ]:
def rank_cryptos(price_df, sentiment_dict):
    ranking_scores = {}
    for crypto in price_df.columns:
        price = price_df[crypto]
        if len(price.dropna()) == 0:
            continue
        sma = calculate_sma(price).iloc[-1]
        rsi = calculate_rsi(price).iloc[-1]
        sentiment = sentiment_dict.get(crypto, 0)
        
        score = (sma/price.iloc[-1])*0.4 + (100-rsi)*0.3 + sentiment*0.3
        ranking_scores[crypto] = score
    ranking = pd.DataFrame(ranking_scores.items(), columns=['Crypto','Score'])
    return ranking.sort_values(by='Score', ascending=False).reset_index(drop=True)

ranking = rank_cryptos(prices, sentiment_scores)
ranking.head(10)

## Step 6: Interactive Visualization

In [ ]:
fig = px.bar(ranking.head(20), x='Crypto', y='Score', title='Top 20 Cryptocurrencies by Weighted Score', color='Score', color_continuous_scale='Viridis')
fig.show()

## Conclusion

- The notebook ranks cryptocurrencies dynamically using market data and sentiment.
- Provides a **data engineering + analytics + visualization** showcase.
- Fully reproducible and interactive for portfolio analysis or further modeling.